In [1]:
import os
import pandas as pd
import glob

In [2]:
#pd.set_option('max_columns', 500)
#pd.set_option('max_colwidth', -1)
pd.set_option('max_colwidth', 300)
#pd.set_option('max_rows', 500)

In [3]:
DATA_IN_DIR = r'/media/rohit/DATA/EverythingD/01SRH-BDBA Acads/Thesis/StoryGenerator/Data/CBTest/data/'
DATA_OUT_DIR = r'/home/rohit/PyWDUbuntu/thesis/CBT/'

In [4]:
os.listdir(DATA_IN_DIR)

['cbtest_CN_test_2500ex.txt',
 'cbtest_CN_train.txt',
 'cbtest_CN_valid_2000ex.txt',
 'cbtest_NE_test_2500ex.txt',
 'cbtest_NE_train.txt',
 'cbtest_NE_valid_2000ex.txt',
 'cbtest_P_test_2500ex.txt',
 'cbtest_P_train.txt',
 'cbtest_P_valid_2000ex.txt',
 'cbtest_V_test_2500ex.txt',
 'cbtest_V_train.txt',
 'cbtest_V_valid_2000ex.txt',
 'cbt_test.txt',
 'cbt_train.txt',
 'cbt_valid.txt',
 'PaxHeader']

In [6]:
def process_and_makes_file_for_gpt2_training(_fin, _fout):
    ## read whole file as dataframe
    df = pd.read_csv(_fin, delimiter='\n', header=None, names=['rawtxt'])
    print(f"Input file = {_fin} \t has {len(df)} lines.")
    
    ## remove the number at start
    df.replace({r'(\d+) (.+)' : r'\2'}, regex=True, inplace=True)

    ## lowercase everything - not doing this for now
    #df['rawtxt'] = dfc['rawtxt'].str.lower()

    ## remove space at second last position just before the end of sentence character
    df.replace({r'(.+)( )(.$)' : r'\1\3'}, regex=True, inplace=True)
    ## remove space before any comma
    df.replace(r' ,' , r',' , regex=True, inplace=True)
    
    ## show counts of hits before any changes
    rep_list = [
        # ain't
        (r"ai n\'t", r'am not'),
        (r"ain\'t", r'am not'),
        (r"ain \'t", r'am not'),

        # aren't
        (r"are n\'t", r'are not'),
        (r"aren\'t", r'are not'),
        (r"aren \'t", r'are not'),

        # can't
        (r"ca n\'t", r'cannot'),
        (r"can\'t", r'cannot'),
        (r"can \'t", r'cannot'),

        # could've
        (r"coul d\'ve", r'could have'),
        (r"could\'ve", r'could have'),
        (r"could \'ve", r'could have'),

        # couldn't
        (r"could n\'t", r'could not'),
        (r"couldn\'t", r'could not'),
        (r"couldn \'t", r'could not'),

        # didn't
        (r"did n\'t", r'did not'),
        (r"didn\'t", r'did not'),
        (r"didn \'t", r'did not'),

        # doesn't
        (r"does n\'t", r'does not'),
        (r"doesn\'t", r'does not'),
        (r"doesn \'t", r'does not'),

        # don't
        (r"do n\'t", r'do not'),
        (r"don\'t", r'do not'),
        (r"don \'t", r'do not'),

        # hadn't
        (r"had n\'t", r'had not'),
        (r"hadn\'t", r'had not'),
        (r"hadn \'t", r'had not'),

        # hasn't
        (r"has n\'t", r'has not'),
        (r"hasn\'t", r'has not'),
        (r"hasn \'t", r'has not'),

        # haven't
        (r"have n\'t", r'have not'),
        (r"haven\'t", r'have not'),
        (r"haven \'t", r'have not'),

        # he'd - also she'd
        (r"h e\'d", r'he would'),
        (r"he\'d", r'he would'),
        (r"he \'d", r'he would'),

        # he'll - also she'll
        (r"h e\'ll", r'he will'),
        (r"he\'ll", r'he will'),
        (r"he \'ll", r'he will'),

        # he's - also she's
        (r"h e\'s", r'he is'),
        (r"he\'s", r'he is'),
        (r"he \'s", r'he is'),

        # how'd
        (r"ho w\'d", r'how did'),
        (r"how\'d", r'how did'),
        (r"how \'d", r'how did'),

        # how'll
        (r"ho w\'ll", r'how will'),
        (r"how\'ll", r'how will'),
        (r"how \'ll", r'how will'),

        # how's
        (r"ho w\'s", r'how is'),
        (r"how\'s", r'how is'),
        (r"how \'s", r'how is'),

        # i'd
        (r"i \'d", r'i would'),
        (r"i\'d", r'i would'),

        # i'll
        (r"i \'ll", r'i will'),
        (r"i\'ll", r'i will'),

        # i'm
        (r"i \'m", r'i am'),
        (r"i\'m", r'i am'),

        # i've
        (r"i \'ve", r'i have'),
        (r"i\'ve", r'i have'),

        # isn't
        (r"is n\'t", r'is not'),
        (r"isn\'t", r'is not'),
        (r"isn \'t", r'is not'),

        # it'd
        (r"i t\'d", r'it would'),
        (r"it\'d", r'it would'),
        (r"it \'d", r'it would'),

        # it'll
        (r"i t\'ll", r'it will'),
        (r"it\'ll", r'it will'),
        (r"it \'ll", r'it will'),

        # it's
        (r"i t\'s", r'it is'),
        (r"it\'s", r'it is'),
        (r"it \'s", r'it is'),

        # let's
        (r"le t\'s", r'let us'),
        (r"let\'s", r'let us'),
        (r"let \'s", r'let us'),

        # ma'am
        (r"m a\'am", r'madam'),
        (r"ma\'am", r'madam'),
        (r"ma \'am", r'madam'),

        # mayn't
        (r"may n\'t", r'may not'),
        (r"mayn\'t", r'may not'),
        (r"mayn \'t", r'may not'),

        # may've
        (r"ma y\'ve", r'may have'),
        (r"may\'ve", r'may have'),
        (r"may \'ve", r'may have'),

        # might've
        (r"migh t\'ve", r'might have'),
        (r"might\'ve", r'might have'),
        (r"might \'ve", r'might have'),

        # mightn't
        (r"migh n\'t", r'might not'),
        (r"mightn\'t", r'might not'),
        (r"mightn \'t", r'might not'),

        # must've
        (r"mus t\'ve", r'must have'),
        (r"must\'ve", r'must have'),
        (r"must \'ve", r'must have'),

        # mustn't
        (r"must n\'t", r'must not'),
        (r"mustn\'t", r'must not'),
        (r"mustn \'t", r'must not'),

        # needn't
        (r"need n\'t", r'need not'),
        (r"needn\'t", r'need not'),
        (r"needn \'t", r'need not'),

        # should've
        (r"shoul d\'ve", r'should have'),
        (r"should\'ve", r'should have'),
        (r"should \'ve", r'should have'),

        # shouldn't
        (r"shouldn \'t", r'should not'),
        (r"shouldn\'t", r'should not'),
        (r"should n\'t", r'should not'),

        # so's
        (r"s o\'s", r'so is'),
        (r"so\'s", r'so is'),
        (r"so \'s", r'so is'),

        # so've
        (r"s o\'ve", r'so have'),
        (r"so\'ve", r'so have'),
        (r"so \'ve", r'so have'),

        # that'd
        (r"tha t\'d", r'that would'),
        (r"that\'d", r'that would'),
        (r"that \'d", r'that would'),

        # that's
        (r"tha t\'s", r'that is'),
        (r"that\'s", r'that is'),
        (r"that \'s", r'that is'),

        # there'd
        (r"ther e\'d", r'there had'),
        (r"there\'d", r'there had'),
        (r"there \'d", r'there had'),

        # there's
        (r"ther e\'s", r'there is'),
        (r"there\'s", r'there is'),
        (r"there \'s", r'there is'),

        # they'd
        (r"the y\'d", r'they would'),
        (r"they\'d", r'they would'),
        (r"they \'d", r'they would'),

        # they'll
        (r"the y\'ll", r'they will'),
        (r"they\'ll", r'they will'),
        (r"they \'ll", r'they will'),

        # they're
        (r"the y\'re", r'they are'),
        (r"they\'re", r'they are'),
        (r"they \'re", r'they are'),

        # they've
        (r"the y\'ve", r'they have'),
        (r"they\'ve", r'they have'),
        (r"they \'ve", r'they have'),

        # wasn't
        (r"was n\'t", r'was not'),
        (r"wasn\'t", r'was not'),
        (r"wasn \'t", r'was not'),

        # we'd
        (r"w e\'d", r'we would'),
        (r"we\'d", r'we would'),
        (r"we \'d", r'we would'),

        # we'll
        (r"w e\'ll", r'we will'),
        (r"we\'ll", r'we will'),
        (r"we \'ll", r'we will'),

        # we're
        (r"w e\'re", r'we are'),
        (r"we\'ve", r'we are'),
        (r"we \'re", r'we are'),

        # we've
        (r"w e\'ve", r'we have'),
        (r"we\'ve", r'we have'),
        (r"we \'ve", r'we have'),

        # weren't
        (r"were n\'t", r'were not'),
        (r"weren\'t", r'were not'),
        (r"weren \'t", r'were not'),

        # what'll
        (r"wha t\'ll", r'what will'),
        (r"what\'ll", r'what will'),
        (r"what \'ll", r'what will'),

        # what're
        (r"w e\'ve", r'we have'),
        (r"we\'ve", r'we have'),
        (r"we \'ve", r'we have'),

        # what's
        (r"wha t\'s", r'what is'),
        (r"what\'s", r'what is'),
        (r"what \'s", r'what is'),

        # what've
        (r"wha t\'ve", r'what have'),
        (r"what\'ve", r'what have'),
        (r"what \'ve", r'what have'),

        # when's
        (r"whe n\'s", r'when is'),
        (r"when\'s", r'when is'),
        (r"when \'s", r'when is'),

        # when've
        (r"whe n\'ve", r'when have'),
        (r"when\'ve", r'when have'),
        (r"when \'ve", r'when have'),

        # where'd
        (r"wher e\'d", r'where did'),
        (r"where\'d", r'where did'),
        (r"where \'d", r'where did'),

        # where's
        (r"wher e\'s", r'where is'),
        (r"where\'s", r'where is'),
        (r"where \'s", r'where is'),

        # where've
        (r"wher e\'ve", r'where have'),
        (r"where\'ve", r'where have'),
        (r"where \'ve", r'where have'),

        # who'll
        (r"wh o\'ll", r'who will'),
        (r"who\'ll", r'who will'),
        (r"who \'ll", r'who will'),

        # who's
        (r"wh o\'s", r'who is'),
        (r"who\'s", r'who is'),
        (r"who \'s", r'who is'),

        # who've
        (r"wh o\'ve", r'who have'),
        (r"who\'ve", r'who have'),
        (r"who \'ve", r'who have'),

        # why's
        (r"wh y\'s", r'why is'),
        (r"why\'s", r'why is'),
        (r"why \'s", r'why is'),

        # why've
        (r"wh y\'ve", r'why have'),
        (r"why\'ve", r'why have'),
        (r"why \'ve", r'why have'),

        # will've
        (r"wil l\'ve", r'will have'),
        (r"will\'ve", r'will have'),
        (r"will \'ve", r'will have'),

        # won't
        (r"wh n\'t", r'will not'),
        (r"won\'t", r'will not'),
        (r"won \'t", r'will not'),

        # would've
        (r"woul d\'ve", r'would have'),
        (r"would\'ve", r'would have'),
        (r"would \'ve", r'would have'),

        # wouldn't
        (r"would n\'t", r'would not'),
        (r"wouldn\'t", r'would not'),
        (r"wouldn \'t", r'would not'),

        # y'all
        (r"y \'all", r'you all'),
        (r"y\'all", r'you all'),

        # you'd
        (r"yo u\'d", r'you had'),
        (r"you\'d", r'you had'),
        (r"you \'d", r'you had'),

        # you'll
        (r"yo u\'ll", r'you all'),
        (r"you\'ll", r'you all'),
        (r"you \'ll", r'you all'),

        # you're
        (r"yo u\'re", r'you are'),
        (r"you\'re", r'you are'),
        (r"you \'re", r'you are'),

        # you've
        (r"yo u\'ve", r'you have'),
        (r"you\'ve", r'you have'),
        (r"you \'ve", r'you have')
    ]
    for val in rep_list:
        rep_this = val[0]
        print(f"""{rep_this} count = {len( df[df['rawtxt'].str.contains(rep_this, regex=True)] )}""")
    
    print(f"\n{'-' * 40}\n")
    ## make changes
    for val in rep_list:
        rep_this , rep_with = val
        df.replace(rep_this, rep_with, regex=True, inplace=True)
        df.replace(rep_this.capitalize(), rep_with.capitalize(), regex=True, inplace=True)
    
    ## create output file
    i = 0
    if os.path.exists(_fout):
        print(f"Aborted, file already exists.")
    else:
        with open(_fout, "w") as f:
            for row in df.itertuples():
                dictRow = row._asdict()
                f.write( dictRow['rawtxt'] + f"\n" )
                i += 1
                #if i > 3:
                #    break
        print(f"Created file :: {_fout}\nWrote {i} lines.\n\n")

In [7]:
## process each .txt file in the folder
for idx, fin in enumerate(glob.glob(DATA_IN_DIR + r'*.txt')):
    root, ext = os.path.splitext(os.path.basename(fin))
    fout = DATA_OUT_DIR + root + r'_OUT' + ext
    print(f"{'*' * 40}\n{'*' * 40}\n")
    print(f"Processing file {idx+1} = {fin}\n")
    process_and_makes_file_for_gpt2_training(fin, fout)

****************************************
****************************************

Processing file 1 = /media/rohit/DATA/EverythingD/01SRH-BDBA Acads/Thesis/StoryGenerator/Data/CBTest/data/cbtest_CN_test_2500ex.txt

Input file = /media/rohit/DATA/EverythingD/01SRH-BDBA Acads/Thesis/StoryGenerator/Data/CBTest/data/cbtest_CN_test_2500ex.txt 	 has 52500 lines.
ai n\'t count = 44
ain\'t count = 0
ain \'t count = 0
are n\'t count = 18
aren\'t count = 0
aren \'t count = 0
ca n\'t count = 188
can\'t count = 0
can \'t count = 0
coul d\'ve count = 0
could\'ve count = 0
could \'ve count = 0
could n\'t count = 162
couldn\'t count = 0
couldn \'t count = 0
did n\'t count = 258
didn\'t count = 0
didn \'t count = 0
does n\'t count = 125
doesn\'t count = 0
doesn \'t count = 0
do n\'t count = 544
don\'t count = 0
don \'t count = 0
had n\'t count = 89
hadn\'t count = 0
hadn \'t count = 0
has n\'t count = 22
hasn\'t count = 0
hasn \'t count = 0
have n\'t count = 81
haven\'t count = 0
haven \'t count = 0


whe n\'ve count = 0
when\'ve count = 0
when \'ve count = 0
wher e\'d count = 0
where\'d count = 0
where \'d count = 23
wher e\'s count = 0
where\'s count = 0
where \'s count = 118
wher e\'ve count = 0
where\'ve count = 0
where \'ve count = 0
wh o\'ll count = 0
who\'ll count = 0
who \'ll count = 44
wh o\'s count = 0
who\'s count = 0
who \'s count = 229
wh o\'ve count = 0
who\'ve count = 0
who \'ve count = 4
wh y\'s count = 0
why\'s count = 0
why \'s count = 0
wh y\'ve count = 0
why\'ve count = 0
why \'ve count = 0
wil l\'ve count = 0
will\'ve count = 0
will \'ve count = 0
wh n\'t count = 0
won\'t count = 0
won \'t count = 0
woul d\'ve count = 0
would\'ve count = 0
would \'ve count = 0
would n\'t count = 10816
wouldn\'t count = 0
wouldn \'t count = 0
y \'all count = 0
y\'all count = 0
yo u\'d count = 0
you\'d count = 0
you \'d count = 3028
yo u\'ll count = 0
you\'ll count = 0
you \'ll count = 5086
yo u\'re count = 0
you\'re count = 0
you \'re count = 3433
yo u\'ve count = 0
you\'ve count

that\'d count = 0
that \'d count = 0
tha t\'s count = 0
that\'s count = 0
that \'s count = 129
ther e\'d count = 0
there\'d count = 0
there \'d count = 0
ther e\'s count = 0
there\'s count = 0
there \'s count = 112
the y\'d count = 0
they\'d count = 0
they \'d count = 35
the y\'ll count = 0
they\'ll count = 0
they \'ll count = 20
the y\'re count = 0
they\'re count = 0
they \'re count = 54
the y\'ve count = 0
they\'ve count = 0
they \'ve count = 7
was n\'t count = 194
wasn\'t count = 0
wasn \'t count = 0
w e\'d count = 0
we\'d count = 0
we \'d count = 2
w e\'ll count = 0
we\'ll count = 0
we \'ll count = 49
w e\'re count = 0
we\'ve count = 0
we \'re count = 21
w e\'ve count = 0
we\'ve count = 0
we \'ve count = 23
were n\'t count = 12
weren\'t count = 0
weren \'t count = 0
wha t\'ll count = 0
what\'ll count = 0
what \'ll count = 0
w e\'ve count = 0
we\'ve count = 0
we \'ve count = 23
wha t\'s count = 0
what\'s count = 0
what \'s count = 25
wha t\'ve count = 0
what\'ve count = 0
what \'ve 

how \'s count = 0
i \'d count = 0
i\'d count = 0
i \'ll count = 0
i\'ll count = 0
i \'m count = 0
i\'m count = 0
i \'ve count = 0
i\'ve count = 0
is n\'t count = 240
isn\'t count = 0
isn \'t count = 0
i t\'d count = 0
it\'d count = 0
it \'d count = 1
i t\'ll count = 0
it\'ll count = 0
it \'ll count = 5
i t\'s count = 0
it\'s count = 0
it \'s count = 212
le t\'s count = 0
let\'s count = 0
let \'s count = 31
m a\'am count = 0
ma\'am count = 0
ma \'am count = 0
may n\'t count = 1
mayn\'t count = 0
mayn \'t count = 0
ma y\'ve count = 0
may\'ve count = 0
may \'ve count = 0
migh t\'ve count = 0
might\'ve count = 0
might \'ve count = 0
migh n\'t count = 0
mightn\'t count = 0
mightn \'t count = 0
mus t\'ve count = 0
must\'ve count = 0
must \'ve count = 0
must n\'t count = 27
mustn\'t count = 0
mustn \'t count = 0
need n\'t count = 41
needn\'t count = 0
needn \'t count = 0
shoul d\'ve count = 0
should\'ve count = 0
should \'ve count = 0
shouldn \'t count = 0
shouldn\'t count = 0
should n\'t cou

Input file = /media/rohit/DATA/EverythingD/01SRH-BDBA Acads/Thesis/StoryGenerator/Data/CBTest/data/cbtest_P_train.txt 	 has 7014630 lines.
ai n\'t count = 10849
ain\'t count = 0
ain \'t count = 0
are n\'t count = 2086
aren\'t count = 0
aren \'t count = 0
ca n\'t count = 52264
can\'t count = 0
can \'t count = 0
coul d\'ve count = 0
could\'ve count = 0
could \'ve count = 0
could n\'t count = 41368
couldn\'t count = 0
couldn \'t count = 0
did n\'t count = 73032
didn\'t count = 0
didn \'t count = 0
does n\'t count = 17264
doesn\'t count = 0
doesn \'t count = 0
do n\'t count = 125169
don\'t count = 0
don \'t count = 0
had n\'t count = 17396
hadn\'t count = 0
hadn \'t count = 0
has n\'t count = 5177
hasn\'t count = 0
hasn \'t count = 0
have n\'t count = 13110
haven\'t count = 0
haven \'t count = 0
h e\'d count = 0
he\'d count = 0
he \'d count = 12025
h e\'ll count = 0
he\'ll count = 0
he \'ll count = 10667
h e\'s count = 0
he\'s count = 0
he \'s count = 28536
ho w\'d count = 0
how\'d count =

wh y\'s count = 0
why\'s count = 0
why \'s count = 0
wh y\'ve count = 0
why\'ve count = 0
why \'ve count = 0
wil l\'ve count = 0
will\'ve count = 0
will \'ve count = 0
wh n\'t count = 0
won\'t count = 0
won \'t count = 0
woul d\'ve count = 0
would\'ve count = 0
would \'ve count = 0
would n\'t count = 320
wouldn\'t count = 0
wouldn \'t count = 0
y \'all count = 0
y\'all count = 0
yo u\'d count = 0
you\'d count = 0
you \'d count = 26
yo u\'ll count = 0
you\'ll count = 0
you \'ll count = 79
yo u\'re count = 0
you\'re count = 0
you \'re count = 24
yo u\'ve count = 0
you\'ve count = 0
you \'ve count = 29

----------------------------------------

Created file :: /home/rohit/PyWDUbuntu/thesis/CBT/cbtest_P_valid_2000ex_OUT.txt
Wrote 42000 lines.


****************************************
****************************************

Processing file 10 = /media/rohit/DATA/EverythingD/01SRH-BDBA Acads/Thesis/StoryGenerator/Data/CBTest/data/cbtest_V_test_2500ex.txt

Input file = /media/rohit/DATA/Ev

the y\'re count = 0
they\'re count = 0
they \'re count = 1442
the y\'ve count = 0
they\'ve count = 0
they \'ve count = 416
was n\'t count = 9349
wasn\'t count = 0
wasn \'t count = 0
w e\'d count = 0
we\'d count = 0
we \'d count = 736
w e\'ll count = 0
we\'ll count = 0
we \'ll count = 3963
w e\'re count = 0
we\'ve count = 0
we \'re count = 857
w e\'ve count = 0
we\'ve count = 0
we \'ve count = 1105
were n\'t count = 833
weren\'t count = 0
weren \'t count = 0
wha t\'ll count = 0
what\'ll count = 0
what \'ll count = 35
w e\'ve count = 0
we\'ve count = 0
we \'ve count = 1105
wha t\'s count = 0
what\'s count = 0
what \'s count = 1234
wha t\'ve count = 0
what\'ve count = 0
what \'ve count = 0
whe n\'s count = 0
when\'s count = 0
when \'s count = 0
whe n\'ve count = 0
when\'ve count = 0
when \'ve count = 0
wher e\'d count = 0
where\'d count = 0
where \'d count = 16
wher e\'s count = 0
where\'s count = 0
where \'s count = 130
wher e\'ve count = 0
where\'ve count = 0
where \'ve count = 0
wh o\'

ma\'am count = 16
ma \'am count = 0
may n\'t count = 3
mayn\'t count = 0
mayn \'t count = 0
ma y\'ve count = 0
may\'ve count = 0
may \'ve count = 0
migh t\'ve count = 0
might\'ve count = 0
might \'ve count = 0
migh n\'t count = 0
mightn\'t count = 0
mightn \'t count = 0
mus t\'ve count = 0
must\'ve count = 0
must \'ve count = 0
must n\'t count = 7
mustn\'t count = 0
mustn \'t count = 0
need n\'t count = 13
needn\'t count = 0
needn \'t count = 0
shoul d\'ve count = 0
should\'ve count = 0
should \'ve count = 0
shouldn \'t count = 0
shouldn\'t count = 0
should n\'t count = 13
s o\'s count = 0
so\'s count = 0
so \'s count = 0
s o\'ve count = 0
so\'ve count = 0
so \'ve count = 0
tha t\'d count = 0
that\'d count = 0
that \'d count = 0
tha t\'s count = 0
that\'s count = 0
that \'s count = 46
ther e\'d count = 0
there\'d count = 0
there \'d count = 0
ther e\'s count = 0
there\'s count = 0
there \'s count = 34
the y\'d count = 0
they\'d count = 0
they \'d count = 10
the y\'ll count = 0
they\'ll

ho w\'ll count = 0
how\'ll count = 0
how \'ll count = 0
ho w\'s count = 0
how\'s count = 0
how \'s count = 0
i \'d count = 0
i\'d count = 0
i \'ll count = 0
i\'ll count = 0
i \'m count = 0
i\'m count = 0
i \'ve count = 0
i\'ve count = 0
is n\'t count = 76
isn\'t count = 0
isn \'t count = 0
i t\'d count = 0
it\'d count = 0
it \'d count = 1
i t\'ll count = 0
it\'ll count = 0
it \'ll count = 2
i t\'s count = 0
it\'s count = 0
it \'s count = 75
le t\'s count = 0
let\'s count = 0
let \'s count = 13
m a\'am count = 0
ma\'am count = 0
ma \'am count = 0
may n\'t count = 1
mayn\'t count = 0
mayn \'t count = 0
ma y\'ve count = 0
may\'ve count = 0
may \'ve count = 0
migh t\'ve count = 0
might\'ve count = 0
might \'ve count = 0
migh n\'t count = 0
mightn\'t count = 0
mightn \'t count = 0
mus t\'ve count = 0
must\'ve count = 0
must \'ve count = 0
must n\'t count = 8
mustn\'t count = 0
mustn \'t count = 0
need n\'t count = 19
needn\'t count = 0
needn \'t count = 0
shoul d\'ve count = 0
should\'ve co